# Effective Python: Chapter 1 - Pythonic Thinking
## Items 5-10

# Item 5: Write Helper Functions Instead of Complex Expressions

Python's pithy syntax makes it easy to write single-line expressions that implement a lot of logic. However, complex expressions can sacrifice readability.

## Example: Parsing URL Query Strings

In [ ]:
from urllib.parse import parse_qs

my_values = parse_qs('red=5&blue=0&green=',
                     keep_blank_values=True)
print(repr(my_values))

In [ ]:
print('Red:     ', my_values.get('red'))
print('Green:   ', my_values.get('green'))
print('Opacity: ', my_values.get('opacity'))

### Problem: Using Boolean Expressions (Too Complex)

In [ ]:
# For query string 'red=5&blue=0&green='
red = my_values.get('red', [''])[0] or 0
green = my_values.get('green', [''])[0] or 0
opacity = my_values.get('opacity', [''])[0] or 0
print(f'Red:     {red!r}')
print(f'Green:   {green!r}')
print(f'Opacity: {opacity!r}')

This is difficult to read and doesn't do everything needed (converting to int).

In [ ]:
# Even worse - wrapping with int()
red = int(my_values.get('red', [''])[0] or 0)
# This is extremely hard to read!

### Using if/else (Better but verbose)

In [ ]:
green_str = my_values.get('green', [''])
if green_str[0]:
    green = int(green_str[0])
else:
    green = 0

### Solution: Helper Function (Best)

In [ ]:
def get_first_int(values, key, default=0):
    found = values.get(key, [''])
    if found[0]:
        return int(found[0])
    return default

In [ ]:
# Much clearer!
green = get_first_int(my_values, 'green')
print(f'Green: {green}')

### Things to Remember

- Python's syntax makes it easy to write single-line expressions that are overly complicated and difficult to read
- Move complex expressions into helper functions, especially if you need to use the same logic repeatedly
- An if/else expression provides a more readable alternative to using Boolean operators or and and in expressions

# Item 6: Prefer Multiple Assignment Unpacking Over Indexing

Python has built-in tuple type for creating immutable, ordered sequences of values.

In [ ]:
snack_calories = {
    'chips': 140,
    'popcorn': 80,
    'nuts': 190,
}
items = tuple(snack_calories.items())
print(items)

### Accessing with Indexes (Traditional)

In [ ]:
item = ('Peanut butter', 'Jelly')
first = item[0]
second = item[1]
print(first, 'and', second)

### Using Unpacking (Pythonic)

In [ ]:
item = ('Peanut butter', 'Jelly')
first, second = item  # Unpacking
print(first, 'and', second)

### Unpacking with Nested Structures

In [ ]:
favorite_snacks = {
    'salty': ('pretzels', 100),
    'sweet': ('cookies', 180),
    'veggie': ('carrots', 20),
}

((type1, (name1, cals1)),
 (type2, (name2, cals2)),
 (type3, (name3, cals3))) = favorite_snacks.items()

print(f'Favorite {type1} is {name1} with {cals1} calories')
print(f'Favorite {type2} is {name2} with {cals2} calories')
print(f'Favorite {type3} is {name3} with {cals3} calories')

### Swapping Values

In [ ]:
def bubble_sort(a):
    for _ in range(len(a)):
        for i in range(1, len(a)):
            if a[i] < a[i-1]:
                a[i-1], a[i] = a[i], a[i-1]  # Swap with unpacking

names = ['pretzels', 'carrots', 'arugula', 'bacon']
bubble_sort(names)
print(names)

### Unpacking in Loops

In [ ]:
snacks = [('bacon', 350), ('donut', 240), ('muffin', 190)]

# Without unpacking (noisy)
for i in range(len(snacks)):
    item = snacks[i]
    name = item[0]
    calories = item[1]
    print(f'#{i+1}: {name} has {calories} calories')

In [ ]:
# With unpacking and enumerate (Pythonic)
for rank, (name, calories) in enumerate(snacks, 1):
    print(f'#{rank}: {name} has {calories} calories')

### Things to Remember

- Python has special syntax called unpacking for assigning multiple values in a single statement
- Unpacking is generalized in Python and can be applied to any iterable, including many levels of iterables within iterables
- Reduce visual noise and increase code clarity by using unpacking to avoid explicitly indexing into sequences

# Item 7: Prefer enumerate Over range

The range built-in function is useful for loops that iterate over a set of integers.

In [ ]:
from random import randint

random_bits = 0
for i in range(32):
    if randint(0, 1):
        random_bits |= 1 << i
        
print(bin(random_bits))

### Iterating Over Lists Directly

In [ ]:
flavor_list = ['vanilla', 'chocolate', 'pecan', 'strawberry']
for flavor in flavor_list:
    print(f'{flavor} is delicious')

### Problem: When You Need the Index

In [ ]:
# Using range (clumsy)
for i in range(len(flavor_list)):
    flavor = flavor_list[i]
    print(f'{i + 1}: {flavor}')

### Solution: Use enumerate

In [ ]:
# enumerate wraps any iterator with a lazy generator
it = enumerate(flavor_list)
print(next(it))
print(next(it))

In [ ]:
# Much clearer with unpacking
for i, flavor in enumerate(flavor_list):
    print(f'{i + 1}: {flavor}')

### Specifying Start Number

In [ ]:
# Start counting from 1 instead of 0
for i, flavor in enumerate(flavor_list, 1):
    print(f'{i}: {flavor}')

### Things to Remember

- enumerate provides concise syntax for looping over an iterator and getting the index of each item from the iterator as you go
- Prefer enumerate instead of looping over a range and indexing into a sequence
- You can supply a second parameter to enumerate to specify the number from which to begin counting (zero is the default)

# Item 8: Use zip to Process Iterators in Parallel

Often you find yourself with many lists of related objects.

In [ ]:
names = ['Cecilia', 'Lise', 'Marie']
counts = [len(n) for n in names]
print(counts)

### Problem: Iterating with range and Indexing

In [ ]:
longest_name = None
max_count = 0

for i in range(len(names)):
    count = counts[i]
    if count > max_count:
        longest_name = names[i]
        max_count = count

print(longest_name)

This is visually noisy. Indexing into arrays happens twice.

### Solution: Use zip

In [ ]:
# zip wraps two or more iterators with a lazy generator
longest_name = None
max_count = 0

for name, count in zip(names, counts):
    if count > max_count:
        longest_name = name
        max_count = count

print(longest_name)

### Warning: zip Truncates to Shortest Iterator

In [ ]:
names.append('Rosalind')
for name, count in zip(names, counts):
    print(name)
# 'Rosalind' is missing!

### Solution: Use zip_longest for Unequal Lengths

In [ ]:
import itertools

for name, count in itertools.zip_longest(names, counts):
    print(f'{name}: {count}')

zip_longest replaces missing values with None (or a fillvalue you specify).

### Things to Remember

- The zip built-in function can be used to iterate over multiple iterators in parallel
- zip creates a lazy generator that produces tuples, so it can be used on infinitely long inputs
- zip truncates its output silently to the shortest iterator if you supply it with iterators of different lengths
- Use the zip_longest function from the itertools built-in module if you want to use zip on iterators of unequal lengths without truncation

# Item 9: Avoid else Blocks After for and while Loops

Python loops have an extra feature: You can put an else block immediately after a loop's repeated interior block.

In [ ]:
for i in range(3):
    print('Loop', i)
else:
    print('Else block!')

Surprisingly, the else block runs immediately after the loop finishes.

### Behavior with break

In [ ]:
for i in range(3):
    print('Loop', i)
    if i == 1:
        break
else:
    print('Else block!')
# Else block is skipped!

### Behavior with Empty Sequences

In [ ]:
for x in []:
    print('Never runs')
else:
    print('For Else block!')

In [ ]:
while False:
    print('Never runs')
else:
    print('While Else block!')

### Example: Finding Coprime Numbers

In [ ]:
a = 4
b = 9

for i in range(2, min(a, b) + 1):
    print('Testing', i)
    if a % i == 0 and b % i == 0:
        print('Not coprime')
        break
else:
    print('Coprime')

### Better: Use Helper Functions

In [ ]:
# First approach: Return early
def coprime(a, b):
    for i in range(2, min(a, b) + 1):
        if a % i == 0 and b % i == 0:
            return False
    return True

assert coprime(4, 9)
assert not coprime(3, 6)
print('Tests passed!')

In [ ]:
# Second approach: Use result variable
def coprime_alternate(a, b):
    is_coprime = True
    for i in range(2, min(a, b) + 1):
        if a % i == 0 and b % i == 0:
            is_coprime = False
            break
    return is_coprime

assert coprime_alternate(4, 9)
assert not coprime_alternate(3, 6)
print('Tests passed!')

### Things to Remember

- Python has special syntax that allows else blocks to immediately follow for and while loop interior blocks
- The else block after a loop runs only if the loop body did not encounter a break statement
- Avoid using else blocks after loops because their behavior isn't intuitive and can be confusing

# Item 10: Prevent Repetition with Assignment Expressions

Assignment expressions (the walrus operator :=) were introduced in Python 3.8 to solve code duplication problems.

In [ ]:
fresh_fruit = {
    'apple': 10,
    'banana': 8,
    'lemon': 5,
}

### Problem: Repetitive Code

In [ ]:
def make_lemonade(count):
    print(f'Making {count} lemonades')

def out_of_stock():
    print('Out of stock!')

# Traditional approach
count = fresh_fruit.get('lemon', 0)
if count:
    make_lemonade(count)
else:
    out_of_stock()

### Solution: Walrus Operator

In [ ]:
# With assignment expression
if count := fresh_fruit.get('lemon', 0):
    make_lemonade(count)
else:
    out_of_stock()

### Assignment Expression in Comparisons

In [ ]:
def make_cider(count):
    print(f'Making {count} ciders')

# Traditional
count = fresh_fruit.get('apple', 0)
if count >= 4:
    make_cider(count)
else:
    out_of_stock()

In [ ]:
# With walrus (note the parentheses)
if (count := fresh_fruit.get('apple', 0)) >= 4:
    make_cider(count)
else:
    out_of_stock()

### Switch/Case Pattern

In [ ]:
def make_smoothies(count):
    print(f'Making smoothies')
    
def slice_bananas(count):
    return count * 4

# Traditional (deeply nested)
count = fresh_fruit.get('banana', 0)
if count >= 2:
    pieces = slice_bananas(count)
    to_enjoy = make_smoothies(pieces)
else:
    count = fresh_fruit.get('apple', 0)
    if count >= 4:
        to_enjoy = make_cider(count)
    else:
        count = fresh_fruit.get('lemon', 0)
        if count:
            to_enjoy = make_lemonade(count)
        else:
            to_enjoy = 'Nothing'

In [ ]:
# With walrus (elegant)
if (count := fresh_fruit.get('banana', 0)) >= 2:
    pieces = slice_bananas(count)
    to_enjoy = make_smoothies(pieces)
elif (count := fresh_fruit.get('apple', 0)) >= 4:
    to_enjoy = make_cider(count)
elif count := fresh_fruit.get('lemon', 0):
    to_enjoy = make_lemonade(count)
else:
    to_enjoy = 'Nothing'
    
print(f'Result: {to_enjoy}')

### Do/While Loop Pattern

In [ ]:
def pick_fruit():
    # Simulate picking fruit (returns empty dict to stop)
    import random
    if random.random() < 0.7:
        return {'apple': 1}
    return {}

def make_juice(fruit, count):
    return [f'{fruit}_juice']

# Traditional (loop-and-a-half)
bottles = []
while True:
    fresh_fruit = pick_fruit()
    if not fresh_fruit:
        break
    for fruit, count in fresh_fruit.items():
        batch = make_juice(fruit, count)
        bottles.extend(batch)
        
print(f'Made {len(bottles)} bottles')

In [ ]:
# With walrus (cleaner)
bottles = []
while fresh_fruit := pick_fruit():
    for fruit, count in fresh_fruit.items():
        batch = make_juice(fruit, count)
        bottles.extend(batch)
        
print(f'Made {len(bottles)} bottles')

### Things to Remember

- Assignment expressions use the walrus operator (:=) to both assign and evaluate variable names in a single expression, thus reducing repetition
- When an assignment expression is a subexpression of a larger expression, it must be surrounded with parentheses
- Although switch/case statements and do/while loops are not available in Python, their functionality can be emulated much more clearly by using assignment expressions